<a href="https://colab.research.google.com/github/dori108/gdg-study/blob/main/6_3_stylegen3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧬 StyleGAN2: 스타일 기반 이미지 생성 네트워크

### 📌 기본 정보

* **제안자**: NVIDIA (2019), "Analyzing and Improving the Image Quality of StyleGAN"
* **핵심 개념**:

  * 전통적인 GAN과 달리, Latent 벡터를 네트워크 각 층에 스타일 형태로 주입
  * `style mixing` 기법으로 한 이미지에 서로 다른 스타일을 결합 가능
  * `truncation trick`으로 생성 이미지 품질 제어

---

## 🛠️ 실습: StyleGAN2로 이미지 생성 및 스타일 믹싱

---

### 🔧 1. 라이브러리 설치 및 예제 다운로드

```python
# StyleGAN2를 위한 nnabla 및 예제 다운로드
!pip install nnabla-ext-cuda120
!git clone https://github.com/sony/nnabla-examples.git
```

---

### 🚀 2. 초기 설정 및 모델 불러오기

```python
# StyleGAN2 폴더로 이동 후 pretrained 모델 다운로드
%cd nnabla-examples/image-generation/stylegan2
!wget https://nnabla.org/pretrained-models/nnabla-examples/GANs/stylegan2/styleGAN2_G_params.h5

# 필요한 모듈 불러오기
from generate import *
from IPython.display import Image, display

# nnabla 설정
dl_context = get_extension_context("cudnn")
nn.set_default_context(dl_context)

# 레이어 수 및 저장 폴더 지정
layer_count = 18
save_dir = 'results'

# 사전학습 파라미터 불러오기
nn.load_parameters("styleGAN2_G_params.h5")
```

---

### 🌱 3. 단일 이미지 생성

#### 🎲 랜덤 설정

```python
base_seed = 228
truncation_psi = 0.23
additive_seed = 195
image_num = 2
```

#### 🖼️ 이미지 생성 및 시각화

```python
random_gen = np.random.RandomState(base_seed)
latent_input = random_gen.randn(image_num, 512)

nn.set_auto_forward(True)
gen_noise = nn.NdArray.from_numpy_array(latent_input)
noise_list = [gen_noise for _ in range(2)]

gen_output = generate(image_num, noise_list, additive_seed, mix_after=7, truncation_psi=truncation_psi)
output_images = convert_images_to_uint8(gen_output, drange=[-1, 1])

for i in range(image_num):
    file_name = f'noise_seed{base_seed}_{i}.png'
    imsave(file_name, output_images[i], channel_first=True)
    display(Image(file_name, width=512, height=512))
```

---

### 🎨 4. 스타일 믹싱을 통한 복합 이미지 생성

#### 🎲 스타일 벡터 선언

```python
primary_seed = 600
secondary_seed = 500
mix_after = 7
auxiliary_seed = 500
truncation_psi = 0.5
rough_batch_size = 2
fine_batch_size = 4
```

#### 🔀 노이즈 벡터 생성

```python
generator_1 = np.random.RandomState(primary_seed)
primary_noise = nn.NdArray.from_numpy_array(generator_1.randn(rough_batch_size, 512))

generator_2 = np.random.RandomState(secondary_seed)
secondary_noise = nn.NdArray.from_numpy_array(generator_2.randn(fine_batch_size, 512))

nn.set_auto_forward(True)
```

#### 🧩 스타일 믹싱 격자 이미지 생성

```python
mixed_images = []
for i in range(rough_batch_size):
    column_images = []
    for j in range(fine_batch_size):
        mixed_noises = [F.reshape(primary_noise[i], (1, 512)), F.reshape(secondary_noise[j], (1, 512))]
        output_rgb = generate(1, mixed_noises, auxiliary_seed, mix_after, truncation_psi)
        column_images.append(convert_images_to_uint8(output_rgb, drange=[-1, 1])[0])
    column_images = np.concatenate(column_images, axis=2)
    mixed_images.append(column_images)

mixed_images = np.concatenate(mixed_images, axis=1)

# Primary row 생성
noises_primary = [primary_noise, primary_noise]
output_primary = generate(rough_batch_size, noises_primary, auxiliary_seed, mix_after, truncation_psi)
primary_image = convert_images_to_uint8(output_primary, drange=[-1, 1])
primary_image = np.concatenate(primary_image, axis=1)

# Secondary column 생성
noises_secondary = [secondary_noise, secondary_noise]
output_secondary = generate(fine_batch_size, noises_secondary, auxiliary_seed, mix_after, truncation_psi)
secondary_image = convert_images_to_uint8(output_secondary, drange=[-1, 1])
secondary_image = np.concatenate(secondary_image, axis=2)

# 격자 구성
blank_image = 255 * np.ones(output_secondary[0].shape).astype(np.uint8)
combined_top_image = np.concatenate((blank_image, secondary_image), axis=2)
final_grid_image = np.concatenate((primary_image, mixed_images), axis=2)
final_grid_image = np.concatenate((combined_top_image, final_grid_image), axis=1)

# 결과 저장 및 시각화
imsave("stylegan2_grid.png", final_grid_image, channel_first=True)
display(Image("stylegan2_grid.png", width=256*(fine_batch_size+1), height=256*(rough_batch_size+1)))
```

---

## 📚 참고 요약

| 항목                          | 설명                                   |
| --------------------------- | ------------------------------------ |
| `latent_input`              | 랜덤 시드 기반으로 생성된 잠재 벡터                 |
| `truncation_psi`            | 생성 이미지 다양성 조절 (낮을수록 평균에 가까운 이미지)     |
| `mix_after`                 | 스타일이 섞이는 layer 기준 (작을수록 저수준 특징부터 섞임) |
| `generate()`                | 이미지 생성 함수                            |
| `convert_images_to_uint8()` | NNabla 형식을 일반 이미지 형식으로 변환            |
| `imsave()`                  | 이미지 저장 함수                            |


## 📌 StyleGAN2 실습에서 꼭 함께 이해하면 좋은 심화 개념 (추가 설명 전용)

---

### 1. 🎛️ **Truncation Trick**의 의미와 역할

* **정의**: latent 벡터 z를 평균(`w_avg`)에 가까운 값으로 보정해, 더 "정제된" 이미지를 생성하는 기법
* **수식적 표현**:

  ```
  w' = w_avg + psi * (w - w_avg)
  ```
* **파라미터 `truncation_psi` 역할**:

  * 값이 **1에 가까우면**: 다양성 높음, 이미지 품질 낮을 수 있음
  * 값이 **0에 가까우면**: 평균 스타일로 수렴 → 더 **선명하지만 단조로운** 이미지 생성

> 💡 『이미지 처리 바이블』에서는 이미지 품질과 다양성 간 trade-off를 설명하며, **latent space sampling**과 관련된 "노이즈 조절" 개념에서 연결됨.

---

### 2. 🧠 **Style Mixing Regularization**: 왜 스타일을 섞는가?

* **학습 단계에서 도입된 기법**:

  * 서로 다른 latent vector를 네트워크 중간 계층에서 섞어 넣음
  * 목적: **한 vector가 전체 스타일을 독점하지 않도록** 유도 → 더 분산된 학습 가능
* **응용 시**: 실제 사용자 입장에서 하나의 얼굴에 서로 다른 눈+입 스타일 등을 조합할 수 있게 됨

> 📗 『이미지 처리 바이블』과 연결:
>
> * “다중 스타일 융합”은 이미지 합성에서 **국소적 특징 추출 → 전역 표현 혼합**으로 설명됨

---

### 3. 🧩 **Mapping Network (Z → W → W+) 개념**

* StyleGAN2는 `Z space` (입력 latent) → `W space` (비선형 매핑된 공간) → 스타일 벡터를 네트워크 각 층에 주입
* **W+ space** (StyleGAN2 개선점):

  * 각 레이어마다 서로 다른 스타일을 주입할 수 있음
  * 더욱 정교한 제어 가능 (예: 코만 수정, 머리 모양만 바꾸기 등)

> 📘 『이미지 처리 바이블』의 GAN 챕터에서는 Latent 공간의 의미를 설명하며,
>
> * \*\*"의미 기반 latent vector 조작"\*\*을 소개.
> * 즉, 얼굴에서 "나이 증가", "웃는 표정"처럼 **의미적 방향**을 latent space에서 직접 조작 가능

---

### 4. 🧬 **이미지 생성 관점에서 StyleGAN2의 차별점**

| 항목              | 기존 GAN           | StyleGAN2                        |
| --------------- | ---------------- | -------------------------------- |
| 입력 latent 사용 방식 | z를 시작부터 직접 주입    | z → w 매핑 후, 각 레이어에 스타일로 주입       |
| 특징 제어 방식        | global latent 하나 | layer-wise 스타일 주입으로 **국소 제어** 가능 |
| 이미지 품질          | artifact 빈도 많음   | 더 정제되고 세밀한 이미지 가능                |
| 다양성 vs. 안정성     | 모델마다 균형 다름       | truncation trick으로 사용자 제어 가능     |